# Benchmarking mace

In [39]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor
import copy
import numpy as np

import sys
import os
import json

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.geom_opt import GeomOpt

current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append(current_dir)

from structure_generation import get_all_configurations_pmg, write_extended_xyz, generate_random_structures

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### AlGaN

In [2]:
AlN_bulk = Structure.from_file('data/bulk_structures/AlN.cif')

supercell_matrix = np.eye(3)*3

AlN_super3 = copy.deepcopy(AlN_bulk)

AlN_super3.make_supercell(supercell_matrix)

AlN_super3.num_sites

/Users/brunocamino/miniconda3/envs/mace-test/lib/python3.11/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


108

In [3]:
# atom_indices_aln = get_all_configurations_pmg(AlN_super3)
# np.savetxt('data/symmetry/aln_108_atom_indices.csv',atom_indices_aln,delimiter=',',fmt='%s')

In [4]:
atom_indices_aln = np.genfromtxt('data/symmetry/aln_108_atom_indices.csv',delimiter=',').astype('int')
atom_indices_aln

array([[  0,   1,   2, ..., 105, 106, 107],
       [ 27,  28,  29, ...,  78,  79,  80],
       [ 18,  19,  20, ...,  87,  88,  89],
       ...,
       [  2,   0,   1, ...,  95,  93,  94],
       [ 29,  27,  28, ...,  77,  75,  76],
       [ 20,  18,  19, ...,  98,  96,  97]])

In [30]:
active_sites=np.where(np.array(AlN_super3.atomic_numbers) == 13)[0]
num_active_sites=len(active_sites)

N_atom = 31

all_config_atom_number = {}

for n,N_atoms in enumerate(np.arange(1,54)):

    structures_random = generate_random_structures(AlN_super3,atom_indices=atom_indices_aln,
                                                   N_atoms=N_atoms,new_species=31,N_config=500,
                                                   DFT_config=20,active_sites=active_sites)

    atom_number_tmp = []
    for structure in structures_random:
        atom_number_tmp.append(list(structure.atomic_numbers))

    all_config_atom_number[str(N_atoms)] = atom_number_tmp

# with open('data/supercell_structures/AlGaN/AlGaN_super3.json', 'w') as json_file:
#     json.dump(all_config_atom_number, json_file)

In [32]:
with open('data/supercell_structures/AlGaN/AlGaN_super3.json', 'r', encoding='utf-8') as json_file:
    AlGaN_super3_all_config = json.load(json_file)
AlGaN_super3_all_config

{'1': [[13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   31,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7,
   7]],
 '2': [[13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   31,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,
   13,

In [38]:
# Generate the Extended XYZ files

lattice = AlN_super3.lattice.matrix
positions = AlN_super3.frac_coords
for N_atoms in AlGaN_super3_all_config.keys():
    
    folder_name = f'data/supercell_structures/AlGaN/AlGaN_super3_{N_atoms}'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    for i,config in enumerate(AlGaN_super3_all_config[N_atoms]):
        structure = Structure(lattice,config,positions)

        write_extended_xyz(structure,os.path.join(folder_name,f'AlGaN_super3_{N_atoms}_{i}.xyz'))

Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_1/AlGaN_super3_1_0.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_0.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_1.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_2.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_3.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_4.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_5.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_6.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_7.xyz'
Extended XYZ file saved as 'data/supercell_structures/AlGaN/AlGaN_super3_2/AlGaN_super3_2_8.xyz'
Extended XYZ file saved as 'da

## mace geometry optimisation

In [ ]:
def mace_geom_opt(atoms):

    atoms_sp = SinglePoint(
        struct=atoms.copy(),
        arch="mace_mp",
        device='cpu',
        calc_kwargs={'model_paths':'small','default_dtype':'float64'},
    )

    atoms_opt = GeomOpt(
        struct=atoms_sp.struct,
        fmax=0.001,
    )

    atoms_opt.run()

    return atoms_opt